# Решение

## Описание выбранного решения

В качестве решения поставленной задачи был выбран подход _RAG_ (_Retrieval Augmented Generation_ — генерация с дополненной выборкой), который соединяет языковую модель с внешней базой знаний. Таким образом ИИ-помощник находит релевантные документы, ранжирует их и генерирует ответ на вопрос пользователя исходя из собственных знаний с опорой на найденные документы.

Для реализации данного подхода были написаны следующие компоненты пайплайна:
- Векторная база знаний с релевантными документами
- Ретривер -- часть, отвечающая за извлечение релевантной информации из базы знаний исходя из запроса пользователя
- Генератор -- часть, отвечающая за управление процессом генерации ответа языковой моделью
- Языковая модель, формирующая ответы на запросы пользователя на основании входных данных из других компонент пайплайна

Таким образом RAG-пайплайн включает в себя следующие шаги:
1. **Индексация** — разрезание документов на чанки, преобразование их в векторное представление и сохранение в базе.
2. **Поиск** — определение нужных чанков по запросу пользователя.
3. **Подготовка** задания — формирование промта, соединяя запрос пользователя с найденными чанками, чтобы языковая модель получила контекст для генерации ответа.
4. **Генерация** — создание ответ с помощью языковой модели.
5. **Проверка** — контроль качества ответа, проверка его точности и отсутствия галлюцинаций. По необходимости можно добавить ссылки на источники данных.

Также исходя из задачи, технических возможностей и архитектуры приложения _Wise Task_ был выбран микросервисный подход в архитектуре созданного приложения с разбиением RAG-пайплайна на микросервисы.

В итоге, с учетом описанного выше, получилось реализовать микросервисное приложение состоящее из следующих логических частей:
1. **Qdrant DB** -- векторная база знаний с релевантными документами про теорию графов и приложение _Wise Task_
2. **Qdrant Indexer** -- сервис, который преобразует имеющиеся документы в чанки и выполняет их загрузку в **Qdrant DB**
3. **Qdrant Ingest** -- сервис поиска релевантных документов в **Qdrant DB**
4. **Core Service** -- сервис-оркестратор, управляет потоком данных в системе, взаимодействует с другими сервисами и базами данных, собирая все необходимое для ответа пользователю и обработки обратной связи от него
5. **PostgreSQL Feedbacks** -- база данных для сбора обратной связи от пользователя о качестве ответа ИИ-помощника
6. **llama_cpp** -- сервис на базе библиотеки llama.cpp с развернутой языковой моделью и сервером для приема запросов на генерацию
7. **LLM Service** -- сервис для управления процессом генерации(построение промтов, настройка параметров, оценка метриками, оркестрация запросов на генерацию)

### Обоснование выбора

В качестве вариантов решения рассматривались следующие подходы:

- Fine-tuning
- RAG
- LLM с длинным контекстом
- Классический поиск

### LLM Service

Данный сервис занимается подготовкой к генерации и управляет ее процессом.

Путь запроса на генерацию состоит из следующих этапов:
1. Поступление запроса на генерацию из Core Service`а, который включает вопрос и заданное количество контекстов.
2. Выполняется определения типа вопроса с помощью классификатора.
3. На основании типа вопроса строится промт включающий вопрос, пришедшие контексты и инструкции для модели.
4. На основании типа вопроса задаются параметры генерации.
5. Промт и параметры посылаются на llama_cpp.
6. Модель генерирует ответ и отправляет результат.
7. Выполняется постобработка ответа.
8. Ответ отдается назад Core Service`у, который передает его наверх пользователю.

#### llama.cpp

В качестве языковой модели, используемой для генерации, была выбрана модель Qwen2.5:3B-Instruct, так как лучше всего показала себя в сравнении с другими исследуемыми моделями, что было выяснено экспериментальным путем. На сервере будет разворачиваться модель с большим количеством параметров.

В качестве способа взаимодействия с моделью была выбрана библиотека llama.cpp, так как исходя из технических возможностей и требований к итоговому продукту был сделан упор на автономность и отказ от внешних зависимостей на сторонних сервисах предоставляющих API-ключи для взаимодействия с моделью и их ограничениях. Данная библиотека в свою очередь предоставляет возможность разворачивать модели локально с большим контролем процесса генерации. Также изначально рассматривался аналог -- фреймворк Ollama, но был осуществлен переход на llama.cpp для более точной настройки под конкретное железо и увеличения производительности и скорости генерации.

#### Классификация вопросов

Исходя из предметной области и ТЗ было выделено три типа приходящих вопросов:
- Definition: вопрос, требующий ответа в виде точной формулировки(определение, теорема, лемма и т.д.)
- Explanation: вопрос, требующий развернутого ответа с объяснением
- Wise Task: вопрос про использование Wise Task

Саму классификацию выполняет класс QueryClassifier, который определяет тип с помощью заранее заданных регулярных выражений. Если ни одно из регулярных выражений не подошло, присваивается тип Explanation, как наиболее общий.

In [ ]:
import re
from logger import get_logger
from patterns import Pattern


class QueryClassifier:
    """Classifier for query type detection"""

    def __init__(self):
        """
        Initialize the QueryClassifier.

        Sets up the logger for the classifier.
        """
        self.logger = get_logger(__name__)

    def classify(self, question: str):
        """
        Detects query type based on predefined patterns in the question.

        Analyzes the input question using regular expression patterns to determine
        if it is seeking a 'definition' or an 'explanation'. Defaults to 'explanation'
        if no patterns match.

        Args:
            question (str): The input question to classify.

        Returns:
            Literal['definition', 'explanation']: The detected query type.
        """
        question_lower = question.lower().strip()
        self.logger.debug(f'Classifying question: "{question}"')

        for pattern in Pattern.wisetask_patterns:
            if re.search(pattern, question_lower):
                self.logger.debug(f'Question classified as WISE_TASK: "{question}"')
                return 'wise_task'

        for pattern in Pattern.definition_patterns:
            if re.search(pattern, question_lower):
                self.logger.debug(f'Question classified as DEFINITION: "{question}"')
                return 'definition'

        for pattern in Pattern.explanation_patterns:
            if re.search(pattern, question_lower):
                self.logger.debug(f'Question classified as EXPLANATION: "{question}"')
                return 'explanation'

        self.logger.debug(f'Question classified as EXPLANATION (default): "{question}"')
        return 'explanation'

#### Промт-инжиниринг

Для каждого типа вопроса был составлен промт, чтобы точнее проинструктировать модель и получть более релевантный ответ.

Промт для вопросов типа Wise Task:


Промт для вопросов типа Explanation:


Промт для вопросов типа Definition:

#### Параметры генерации

Были заданы следующие параметры генерации для вопросов типа Definition и Wise Task:

In [ ]:
temp = 0.3
top_p = 0.5
top_k = 20
repeat_penalty = 1.05
num_predict = 96

Были заданы следующие параметры генерации для вопросов типа Explanation:

In [ ]:
temp = 0.5
top_p = 0.7
top_k = 40
repeat_penalty = 1.05
num_predict = 320

Также для каждого из типов вопросов были выбраны заданы следующие параметры:

In [ ]:
seed = 42
num_ctx = 4096

#### Параметры запуска модели. Оптимизация скорости и качества генерации.

Для оптимизации скорости генерации ответов и их качества, были настроены следующие параметры модели:

Производительность модели
- mlock
- flash-attn
- parallel N
- cont-batching

Кэширование запросов
- slot-prompt-similarity
- cache-ram

Улучшение качества ответов в RAG
- no-context-shift

Приведенные выше параметры использовались при локальном тестировании и настройки модели, при развертывании на сервере будут внесены следующие изменения("+" - параметр добавлен, "-" - параметр убран):

- gpu-layers +
- ubatch-size +
- mlock -